In [2]:
import os
print(os.getcwd())
import joblib
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import torch
from torch.nn import Softmax
from torch.utils.data import DataLoader

from notebooks.load_ckpt_w_attn import define_model

import json
import argparse
import time
from itertools import chain
from tqdm import tqdm


/home/hyeryungson/mucoco


In [3]:
import os
from tqdm import tqdm
import numpy as np
import torch
import torch.nn.functional as F
import pandas as pd
import csv

In [4]:
BATCH_SIZE=64
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [5]:
# 원본 데이터 로드
samples=pd.read_csv('./notebooks/results/samples.csv')
# label이 1인 데이터만 사용
sample1=samples.loc[samples['label']==1].copy()

In [6]:
# load trained model
ckpt_path='/home/hyeryungson/mucoco/models_bak_contd/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/checkpoint_best/pytorch_model.bin'
model, config, tokenizer = define_model(ckpt_path)

None
None
look above for padding
Adding special tokens


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2-large and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializin

['<s>', '<pad>', '</s>', '<unk>', 'madeupword0000', 'madeupword0001', 'madeupword0002', '<mask>']
computing vecmap
torch.Size([1280, 768]) torch.Size([768, 1280])


In [7]:
# 논문에서는 gpt-2의 tokenizer를 사용하였으므로, mask token이 기존에는 없었음
tokenizer.all_special_ids, tokenizer.all_special_tokens, tokenizer.vocab_size

# tokenizer에 mask token 추가
SPECIAL_TOKENS = {"mask_token": "<mask>"}
tokenizer.add_special_tokens(SPECIAL_TOKENS)

tokenizer.all_special_ids, tokenizer.all_special_tokens, tokenizer.vocab_size

([50256, 50257], ['<|endoftext|>', '<mask>'], 50257)

In [8]:
test_sent = sample1['text'].tolist()[0:10]
batch = tokenizer(test_sent, padding=True, return_tensors="pt", truncation=True)

In [9]:
classifier_output = model.forward(
batch["input_ids"].cuda(),
attention_mask=batch["attention_mask"].cuda(),
)

In [14]:
# loss? # output itself?
loss = F.cross_entropy(classifier_output[0], torch.ones(len(test_sent)).long().cuda())
loss.backward()

In [16]:
# embedding layer
for n, p in list(model.named_parameters())[:10]:
    print(n, p.shape)

roberta.embeddings.word_embeddings.0.weight torch.Size([50257, 1280])
roberta.embeddings.word_embeddings.1.weight torch.Size([768, 1280])
roberta.embeddings.position_embeddings.weight torch.Size([514, 768])
roberta.embeddings.token_type_embeddings.weight torch.Size([1, 768])
roberta.embeddings.LayerNorm.weight torch.Size([768])
roberta.embeddings.LayerNorm.bias torch.Size([768])
roberta.encoder.layer.0.attention.self.query.weight torch.Size([768, 768])
roberta.encoder.layer.0.attention.self.query.bias torch.Size([768])
roberta.encoder.layer.0.attention.self.key.weight torch.Size([768, 768])
roberta.encoder.layer.0.attention.self.key.bias torch.Size([768])


In [ ]:
### hidden state return

In [20]:
norm = torch.norm(list(model.named_parameters())[1][1].grad, dim=-1)
norm = torch.where(norm > 0, norm, torch.full_like(norm, 1e-10))

In [22]:
norm.shape

torch.Size([768])

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
np.random.seed(args.seed)
torch.manual_seed(args.seed)

K = 1
rbt_config = RobertaConfig.from_pretrained(args.rbt_path, cache_dir=None)
rbt_tknzr = RobertaTokenizer.from_pretrained(args.rbt_path, do_lower_case=False)
rbt_model = RobertaForMTL.from_pretrained(args.rbt_path, config=rbt_config,
                                            task_names=['bertscore', 'maskedlm', 'cls'])
rbt_model.to(device)
oreo = OREO(rbt_model=rbt_model, rbt_tknzr=rbt_tknzr, k=K, device=device)
data = pd.read_csv(args.infile, sep='\n', quoting=csv.QUOTE_NONE, header=None)
results = []

for orgs in tqdm(batchify(data[0].tolist(), args.batch_size), total=len(data[0])/args.batch_size, desc='loading input'):
    torch.cuda.empty_cache()
    cand_inps_sents = orgs.copy()
    edit_track = [[] for _ in range(len(orgs))]

    for step in range(args.iter_step):
        torch.cuda.empty_cache()

        inps = [rbt_tknzr.tokenize(sent, add_prefix_space=True)[:50] for sent in cand_inps_sents]

        if args.attribute == 'formality':
            abbr_pos = oreo.select_abbr_span(inps)

        cand_inps, cand_lens, _, _ = collate_no_tokenize(inps, rbt_tknzr.convert_tokens_to_ids, device=device)
        bsz, seqlen = cand_inps.size()
        oreo.model.output_hidden_states = True
        attr_val_org, hid_states_org = oreo.cal_attr(cand_inps, hook_hid_grad=True)
        oreo.model.output_hidden_states = False

        loss = F.cross_entropy(attr_val_org, torch.ones(bsz).long().to(device))
        loss.backward()

        attr_val = F.softmax(attr_val_org, dim=1).cpu()
        del attr_val_org
        attr_val_mask = torch.where(attr_val[:, 1] > args.cls_thld, torch.zeros(bsz),
                                    torch.ones(bsz)).bool().tolist()

        attr_scores = attr_val[:, 1].tolist()
        for i, (sent, attr_score) in enumerate(zip(cand_inps_sents, attr_scores)):
            ex = {'sent': sent, 'score': attr_score}
            edit_track[i].append(ex)

        ent_mask = get_entity_mask(cand_inps_sents, cand_inps, rbt_tknzr, add_prefix_space=True)
        grad_mask = (cand_inps.ne(oreo.pad_idx)) * (cand_inps.ne(oreo.bos_idx)) * \
                    (cand_inps.ne(oreo.eos_idx)) * ent_mask

        del cand_inps, ent_mask

        for i, state in enumerate(hid_states_org):
            norm = torch.norm(state.grad, dim=-1).unsqueeze(2)
            norm = torch.where(norm > 0, norm, torch.full_like(norm, 1e-10))
            if i == 0:
                max_span_len = torch.floor((cand_lens - 2) * args.max_mask_ratio)
                max_span_len = torch.where(max_span_len < 1, torch.ones_like(max_span_len), max_span_len)
                emb_ngram_top1 = get_ngram_topk(norm.squeeze(-1), grad_mask, args.C, max_span_len, args.fixed_span_len)
                break

        del grad_mask, hid_states_org
        oreo.model.zero_grad()

        if args.attribute == 'formality':
            ins_pos_l = [item if item else emb_ngram_top1[i] for i, item in enumerate(abbr_pos)]
        elif args.attribute == 'simplicity':
            ins_pos_l = emb_ngram_top1

        cand_inps, cand_lens, _, ins_pos = collate_inp_mask_after_span(inps, rbt_tknzr.convert_tokens_to_ids,
                                                                        ins_pos_l,
                                                                        device=device)

In [ ]:
def get_ngram_topk(inps_grad, mask, C, max_span_len, fixed_span_len=3):
    inps_grad = torch.where(mask.eq(1), inps_grad, torch.full_like(inps_grad, -1e16))
    bsz, seqlen = inps_grad.size()
    
    ngrams_b = [[] for _ in range(bsz)]
    for ngram in range(1, fixed_span_len + 1):
        ngram_grad = 0.
        for i in range(ngram):
            ngram_grad += inps_grad[:, i: i + (seqlen - ngram + 1)]
        
        value, position = torch.topk(ngram_grad / (ngram + C), 1, dim=1)
        for bat, (val, pos) in enumerate(zip(value.cpu().tolist(), position.cpu().tolist())):
            for v, p in zip(val, pos):
                if v > 0:
                    ngrams_b[bat].append((v, list(range(p, p + ngram))))
                else:
                    assert ValueError, "Value of topk grad cannot be negative"

    sorted_ngrams_b = []
    for bat, seq in enumerate(ngrams_b):
        seq = [val_idx for val_idx in seq if len(val_idx[1]) <= max_span_len[bat]]
        sorted_ngrams_b.append(sorted(seq, key=lambda x: x[0], reverse=True)[0][1])

    return sorted_ngrams_b